In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [10]:
from os import listdir
from os.path import isfile, join
posFiles = [f for f in listdir("/Users/brandonbryant/deeplearning/pos/") if isfile(join("/Users/brandonbryant/deeplearning/pos/", f))]
negFiles = [f for f in listdir("/Users/brandonbryant/deeplearning/neg/") if isfile(join("/Users/brandonbryant/deeplearning/neg/", f))]

In [79]:
#Read into data frame and have 2 columns, text and sentiment

#List of strings of positve files
files = list()
sentiments = list()
for pos in posFiles:
    #file to string 
    with open ("/Users/brandonbryant/deeplearning/pos/" + pos, "r") as myfile:
        file=myfile.readlines()
        files.append(file)
        sentiments.append(1)
    
for neg in negFiles:
    #file to string
     with open ("/Users/brandonbryant/deeplearning/neg/" + neg, "r") as myfile:
        file=myfile.readlines()
        files.append(file)
        sentiments.append(0)
    


In [80]:
stopwordsRemoved = list()
from nltk.corpus import stopwords
for doc in files:
    filtered_words = [word for word in doc if word not in stopwords.words('english')]
    stopwordsRemoved.append(filtered_words)

In [93]:
tokenizer = Tokenizer( split=' ')
Xvects = []
for i in range(0,len(data['text'])):
    tokenizer.fit_on_texts(data['text'][i])
    X = tokenizer.texts_to_sequences(data['text'][i])
    X = pad_sequences(X)
    Xvects.append(X)



In [82]:
#Don't run 

#data = pd.DataFrame(
 #   {'text': files,
 #    'sentiment': sentiments,
 #   })

#max_fatures = 2000
#tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
#tokenizer.fit_on_texts(data['text'].values)
#X = tokenizer.texts_to_sequences(data['text'].values)
#X = pad_sequences(X)

In [94]:
yVects = sentiments

In [95]:
import pandas as pd
df = pd.DataFrame(
    {'Y': yVects,
     'x': Xvects,
    })

In [96]:
from sklearn.utils import shuffle
df = shuffle(df)

In [101]:
df.head()

,Y,x
191,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
816,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
42,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1808,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
44,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [102]:
counter = 0
x_train = []
y_train = []
x_test = []
y_test = []

for index, row in df.iterrows():
    #add to train
    if counter < 1600:
        x_train.append(row['x'])
        y_train.append(row['Y'])
    else:
        x_test.append(row['x'])
        y_test.append(row['Y'])
    counter = counter+1

In [111]:
type(x_train)


list

In [112]:
from keras.datasets import imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data()


17465344/17464789 [==============================] - 1s 0us/step


In [117]:
from keras.preprocessing import sequence
x_train = sequence.pad_sequences(x_train, maxlen=500)
x_test = sequence.pad_sequences(x_test, maxlen=500)

In [104]:
embed_dim = 128
lstm_out = 196


max_fatures =2000
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/Users/brandonbryant/anaconda/envs/deeplearning/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  import sys
/Users/brandonbryant/anaconda/envs/deeplearning/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 29, 128)           256000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [118]:
batch_size = 32
model.fit(x_train, y_train, nb_epoch = 7, batch_size=batch_size, verbose = 2)

/Users/brandonbryant/anaconda/envs/deeplearning/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ValueError: Error when checking input: expected embedding_3_input to have shape (29,) but got array with shape (500,)